In [ ]:
import requests # type: ignore
import numpy as np # type: ignore
import re
import json
import pandas as pd # type: ignore
import os
import time
from datetime import datetime
import asyncio
import aiohttp

In [ ]:
NUM_COHORTS = 24
COHORT_SIZE = 8
NUM_ROUNDS = 75

OUTPUT_DIR = "out4"
MEMORY_DIR =f"{OUTPUT_DIR}/memory"
MEMORY_SIZE = 300
MODEL_NAME= "mistral"
EXPERIMENT_RESULTS = f"{OUTPUT_DIR}/experiment.csv"
COHORT_RESULTS = f"{OUTPUT_DIR}/cohorts.txt"
LOGS_PATH = f"{OUTPUT_DIR}/logs/log_{datetime.now().strftime("%Y%m%d%H%M%S")}.txt"

"""
	0: Only Reasoning
	1: No Reasoning
	2: Reasoning and Risk Aversion
	3: Reasoning and Risk 10
	4: Reasoning and Risk 1
	5: Reasoning and Risk 5
"""
PROMPT_TYPE = 2
LLM_EVAL = True

In [ ]:
# NUM_COHORTS = 6
# COHORT_SIZE = 4
# NUM_ROUNDS = 3

In [ ]:
TREATMENTS = {
	0.6: {
		('R', 'R'): (45, 45),
		('R', 'B'): (0, 42),
		('B', 'R'): (42, 0),
		('B', 'B'): (12, 12)
	},
	1: {
		('R', 'R'): (45, 45),
		('R', 'B'): (0, 40),
		('B', 'R'): (40, 0),
		('B', 'B'): (20, 20)
	},
	2: {
		('R', 'R'): (45, 45),
		('R', 'B'): (0, 35),
		('B', 'R'): (35, 0),
		('B', 'B'): (40, 40)
	}
}

In [ ]:
RISK_PREF = {
	0: "",
	1: "You avoid taking any risks and prefer to stay in familiar and safe situations. You value security and certainty in all aspects of your life. ",
	2: "You are highly cautious and generally avoid taking risks. You need substantial reassurance and certainty before making decisions. ",
	3: "You are very careful and tend to avoid risks. You prefer to take a conservative approach and avoid uncertainties. ",
	4: "You are somewhat careful and tend to avoid taking risks unless absolutely necessary. You weigh the pros and cons thoroughly before making decisions. ",
	5: "You are moderately cautious and may take risks but generally prefer to avoid them. You like to have some degree of security and predictability. ",
	6: "You have a balanced approach to risk. You are willing to take risks occasionally, but you also value security and caution in equal measure. ",
	7: "You are fairly open to taking risks but still prefer to assess the situation carefully. You are not reckless, but you are not overly cautious either. ",
	8: "You are willing to take risks and are fairly comfortable with uncertainty. You tend to embrace new opportunities even if they involve some risk. ",
	9: "You are very open to taking risks and often seek out new and challenging opportunities. You are comfortable with uncertainty and potential failure. ",
	10: "You are extremely comfortable taking risks and are always ready to embrace new challenges, even if they involve significant uncertainty or potential loss. "
}

RISK_SPLIT = {
	1: 4.91,
	2: 7.81,
	3: 11.86,
	4: 10.97,
	5: 15.75,
	6: 15.16,
	7: 15.35,
	8: 9.79,
	9: 4.65,
	10: 3.76
}

In [ ]:
API_URL = "http://127.0.0.1:11434"
GENERATE_ENDPOINT = "/api/generate"
CHAT_ENDPOINT = "/api/chat"

In [ ]:
if not os.path.exists(OUTPUT_DIR):
	os.makedirs(OUTPUT_DIR)

if not os.path.exists(f"{OUTPUT_DIR}/{LOGS_PATH.split('/')[1]}"):
	os.makedirs(f"{OUTPUT_DIR}/{LOGS_PATH.split('/')[1]}")

rand_gen = np.random.default_rng(seed=23)

In [ ]:
class Timer:
	def __init__(self):
		self.start_time = None
		self.end_time = None

	def start(self):
		self.start_time = time.time()
		return self.start_time
	
	def end(self):
		self.end_time = time.time()
		return self.elapsed()

	def elapsed(self):
		return f"{(self.end_time - self.start_time):.3f}s"

In [ ]:
class Participant:
	def __init__(self, id):
		self.id = id
		self.memory = []
		self.payout = 0
		self.rounds = 0
		self.risk_preference = 0
		self.prompt = ""

	def __eq__(self, other):
		return self.id == other.id
	
	def __lt__(self, other):
		return self.id < other.id
	
	def __gt__(self, other):
		return self.id > other.id
	
	def __hash__(self):
		return int(self.id)
	
	def __repr__(self):
		return f"<Participant {{id: {self.id}}}>"
	
	def set_prompt(self, prompt):
		self.prompt = prompt
	
	def set_risk(self, pref):
		self.risk_preference = pref

	def load_memory(self, ctx):
		self.memory = ctx
	
	def update_memory(self, ctx):
		self.memory = [*self.memory, *ctx[-2:]]
		self.save_memory()
		
	def save_memory(self):
		if not os.path.exists(MEMORY_DIR):
			os.makedirs(MEMORY_DIR)
		with open(f"{MEMORY_DIR}/{self.id}.json", "wt") as file:
			file.write(json.dumps(self.memory, indent=4))

In [ ]:
class Cohort:
	def __init__(self, id, participants):
		self.id = id
		self.participants = participants
		self.treatment = None

In [ ]:
def write_and_print(filepath, data, end="\n"):
	with open(filepath, "a") as file:
		file.write(str(data) + end)
	print(data, end=end)

In [ ]:
def generate_cohorts():
	participant_ids = np.arange(101, NUM_COHORTS * COHORT_SIZE + 101, dtype=int)
	rand_gen.shuffle(participant_ids)
	participants = [Participant(id) for id in participant_ids]
	
	if PROMPT_TYPE == 2:
		temp = 0
		for risk, per in RISK_SPLIT.items():
			temp += per
			for participant in participants[int(len(participants) * (temp - per) / 100):int(len(participants) * temp / 100)]:
				participant.set_risk(risk)
		
		for participant in participants[int(len(participants) * temp / 100):]:
			risk = rand_gen.integers(1, 11)
			participant.set_risk(risk)

		rand_gen.shuffle(participants)
	elif PROMPT_TYPE == 3 or PROMPT_TYPE == 4 or PROMPT_TYPE == 5:
		risk = 10 if PROMPT_TYPE == 3 else 1 if PROMPT_TYPE == 4 else 5
		for participant in participants:
			participant.set_risk(risk)

	cohort_ids = np.arange(1, NUM_COHORTS + 1, dtype=int)
	cohort_splits = np.array_split(participants, NUM_COHORTS)
	cohorts = [Cohort(id, participant) for id, participant in zip(cohort_ids, cohort_splits)]


	treatment_splits = np.array_split(cohorts, len(TREATMENTS))
	for cohorts_for_treatment, treatment in zip(treatment_splits, TREATMENTS.keys()):
		for cohort in cohorts_for_treatment:
			cohort.treatment = treatment

	return cohorts

In [ ]:
async def prompt_model(player):
	prompt = player.prompt
	context = player.memory
	timer = Timer()
	
	messages = [
		*context,
		{
			"role": "user",
			"content": prompt
		}
	]
	messages = [messages[0], *messages[-(MEMORY_SIZE - 1):]]
	if PROMPT_TYPE == 1:
		messages.insert(0, {
			"role": "system",
			"content": "You are an undergraduate student participating in a lab experiment. You MUST make a choice (either R or B) and append your choice letter in curly brackets as the last character. Answer in less than 5 words with no reasoning and only the choice in curly braces."
		})
	else:
		messages.insert(0, {
			"role": "system",
			"content": "You are an undergraduate student participating in a lab experiment. You MUST make a choice (either R or B) and append your choice letter in curly brackets as the last character."
		})
	data = {
    "model": MODEL_NAME,
		"messages": messages,
    "stream": False
	}

	async with aiohttp.ClientSession() as session:
		write_and_print(LOGS_PATH, f"Prompting participant {player.id}")
		timer.start()
		async with session.post(API_URL + CHAT_ENDPOINT, json=data) as response:
			res = await response.json()
			timer.end()
			write_and_print(LOGS_PATH, f"Got response for participant {player.id} in {timer.elapsed()}")
			return [*messages, res['message']]

In [ ]:
def evaluate_response(response):
	timer = Timer()

	data = {
    "model": MODEL_NAME,
		"system": "You are an assistant tasked with finding the choice (either R or B) chosen by a player. You will be given the player's response. Answer in less than 5 words with only the choice (either R or B) in curly braces.",
		"prompt": response,
    "stream": False
	}

	write_and_print(LOGS_PATH, f"Prompting Evaluator for choice")
	timer.start()
	res = requests.post(API_URL + GENERATE_ENDPOINT, json=data).json()
	timer.end()
	write_and_print(LOGS_PATH, f"Got response in {timer.elapsed()}")
	
	return res['response']

In [ ]:
def get_choice(response, eval=LLM_EVAL):
	if eval:
		org_response = response
		response = evaluate_response(response)
	matches = re.findall("{[RB]}", response)
	if len(matches) > 0:
		return matches[-1][1]
	else:
		matches = re.findall("[RB]", response)
		if len(matches) > 0:
			return matches[-1]
		else:
			raise Exception(f"Could not find choice in response: {response}")

In [ ]:
async def play_round(participant, opponent):
	ctx1, ctx2 = await asyncio.gather(
		asyncio.create_task(prompt_model(participant)),
		asyncio.create_task(prompt_model(opponent))
	)
	
	return ctx1, ctx2

In [ ]:
def generate_start_prompt(treatment, risk):
	return f"You are an undergraduate student participating in a lab experiment. {RISK_PREF[risk]}You play a game with an anonymous player in which you simultaneously make a choice. Your payoff depends on both choices. If you both pick R, you each get {TREATMENTS[treatment][('R','R')][0]}$. If you choose R while they choose B, you get {TREATMENTS[treatment][('R','B')][0]}$, and they get {TREATMENTS[treatment][('R','B')][1]}$. Similarly, if you pick B while they pick R, you get {TREATMENTS[treatment][('B','R')][0]}$, and they get {TREATMENTS[treatment][('B','R')][1]}$. If you both pick B, you each earn {TREATMENTS[treatment][('B','B')][0]}$. The game has {NUM_ROUNDS} rounds. What's your choice? Perform reasoning as a human player. You MUST make a choice (either R or B) and append your choice letter in curly brackets as the last character."

In [ ]:
def generate_cont_prompt(choice1, choice2, treatment):
	return f"You chose {choice1}. Your opponent chose {choice2}. Your payoff is {TREATMENTS[treatment][(choice1, choice2)][0]}$. Your opponent's payoff is {TREATMENTS[treatment][(choice1, choice2)][1]}$. Please play the next round."

In [ ]:
def generate_end_prompt(choice1, choice2, treatment, payout):
	return f"You chose {choice1}. Your opponent chose {choice2}. Your payoff is {TREATMENTS[treatment][(choice1, choice2)][0]}$. Your opponent's payoff is {TREATMENTS[treatment][(choice1, choice2)][1]}$. The game is now over. Your total payoff was {payout + TREATMENTS[treatment][(choice1, choice2)][0]}$."

In [ ]:
def generate_game_order(cohort):
	result = []
	participants = cohort.participants

	for _ in range(NUM_ROUNDS):
		pairs = []
		remaining_participants = participants.copy()

		while len(remaining_participants) > 1:
			i, j = rand_gen.choice(len(remaining_participants), size=2, replace=False)
			pair = (remaining_participants[i], remaining_participants[j])
			pairs.append(pair)

			remaining_participants = list(remaining_participants)
			remaining_participants.remove(pair[0])
			remaining_participants.remove(pair[1])

		if len(remaining_participants) == 1:
			pairs.append((remaining_participants[0], None))

		result.extend(pairs)

	return result

In [ ]:
def save_results(filename, row):
	new_row = pd.DataFrame(row)
	if os.path.isfile(filename):
		data = pd.read_csv(filename)
		data = pd.concat([data, new_row])
		data.to_csv(filename, index=False)
	else:
		data = new_row
		data.to_csv(filename, index=False)

In [ ]:
game_id = 0
total_rounds = NUM_COHORTS * COHORT_SIZE * NUM_ROUNDS // 2

timer = Timer()

cohorts = generate_cohorts()

In [ ]:
with open(COHORT_RESULTS, "w+") as file:
	file.write("")

write_and_print(COHORT_RESULTS, "----*----"*5)
for cohort in cohorts:
	write_and_print(COHORT_RESULTS, f"Cohort ID: {cohort.id}", end="\t")
	write_and_print(COHORT_RESULTS, f"Cohort Treatment: {cohort.treatment}")
	write_and_print(COHORT_RESULTS, "Cohort Participants:")
	for participant in cohort.participants:
		write_and_print(COHORT_RESULTS, participant.id, end="\t")
	write_and_print(COHORT_RESULTS, "")
	write_and_print(COHORT_RESULTS, "----*----"*5)

In [ ]:
with open(LOGS_PATH, "w+") as file:
	file.write("")
	
while (len(pd.read_csv(EXPERIMENT_RESULTS)) if os.path.exists(EXPERIMENT_RESULTS) else -1) < total_rounds:
	try:
		for cohort in cohorts:
			if os.path.exists(EXPERIMENT_RESULTS):
				results = pd.read_csv(EXPERIMENT_RESULTS)
				if len(results[results['cohort_id'] == cohort.id]) == NUM_ROUNDS * COHORT_SIZE // 2:
					write_and_print(LOGS_PATH, f"Skipping cohort {cohort.id}")
					game_id += len(generate_game_order(cohort))
					continue
				elif results[results['cohort_id'] == cohort.id].shape[0] < NUM_ROUNDS * COHORT_SIZE // 2:
					results = results[results['cohort_id'] != cohort.id]
					results.to_csv(EXPERIMENT_RESULTS, index=False)
					cohort_members = [f"{participant.id}.json" for participant in cohort.participants]
					for member in cohort_members:
						if os.path.exists(f"{MEMORY_DIR}/{member}"):
							os.remove(f"{MEMORY_DIR}/{member}")
					game_id = results.shape[0]
					write_and_print(LOGS_PATH, f"Resetting game_id to {game_id}")
			order = generate_game_order(cohort)
			for participant, opponent in order:
				game_id += 1
				write_and_print(LOGS_PATH, f"Cohort: {cohort.id} - Treatment: {cohort.treatment} - Game {game_id}/{total_rounds} ({(game_id/total_rounds)*100:.3f}%): {participant.id} vs {opponent.id}")

				if participant.rounds == 0:
					write_and_print(LOGS_PATH, f"First round of participant {participant.id}")
					participant.set_prompt(generate_start_prompt(cohort.treatment, participant.risk_preference))

				if opponent.rounds == 0:
					write_and_print(LOGS_PATH, f"First round of participant {opponent.id}")
					opponent.set_prompt(generate_start_prompt(cohort.treatment, opponent.risk_preference))
				
				retry = True
				while retry:
					try:
						ctx1, ctx2 = await play_round(participant, opponent)

						choice1 = get_choice(ctx1[-1]['content'])
						choice2 = get_choice(ctx2[-1]['content'])

						retry = False
					except Exception as e:
						write_and_print(LOGS_PATH, "=====***====="*5)
						write_and_print(LOGS_PATH, f"Error: {e}")
						write_and_print(LOGS_PATH, "=====***====="*5)
					

				participant.update_memory(ctx1)
				opponent.update_memory(ctx2)

				participant.rounds += 1
				opponent.rounds += 1

				participant.payout = TREATMENTS[cohort.treatment][(choice1, choice2)][0]
				opponent.payout = TREATMENTS[cohort.treatment][(choice1, choice2)][1]

				write_and_print(LOGS_PATH, f"Saving results to file {EXPERIMENT_RESULTS}")
				save_results(EXPERIMENT_RESULTS, {'game_id': [game_id], 'treatment': [cohort.treatment], 'cohort_id': [cohort.id], 'round': [participant.rounds], 'player1_id': [participant.id], 'player2_id': [opponent.id], 'player1_choice': [choice1], 'player2_choice': [choice2], 'player1_payout': [participant.payout], 'player2_payout': [opponent.payout]})
				write_and_print(LOGS_PATH, f"Saved results to file")

				if participant.rounds == NUM_ROUNDS:
					participant.set_prompt(generate_end_prompt(choice1, choice2, cohort.treatment, participant.payout))
				else:
					participant.set_prompt(generate_cont_prompt(choice1, choice2, cohort.treatment))

				if opponent.rounds == NUM_ROUNDS:
					opponent.set_prompt(generate_end_prompt(choice1, choice2, cohort.treatment, opponent.payout))
				else:
					opponent.set_prompt(generate_cont_prompt(choice2, choice1, cohort.treatment))
				
				write_and_print(LOGS_PATH, '----*----'*10)
	except Exception as e:
		write_and_print(LOGS_PATH, "=====***====="*5)
		write_and_print(LOGS_PATH, f"Error: {e}")
		write_and_print(LOGS_PATH, "=====***====="*5)
		LOGS_PATH = f"{OUTPUT_DIR}/logs/log_{datetime.now().strftime("%Y%m%d%H%M%S")}.txt"
		continue

write_and_print(LOGS_PATH, "Experiment completed successfully!")